In [10]:
import numpy as np
import tensorflow as tf
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from PIL import Image

### Input 8x8

In [11]:
# # Load dataset
# digits = load_digits()
# X = digits.images / 16.0
# y = digits.target.reshape(-1, 1)

# # One-hot encode labels
# encoder = OneHotEncoder(sparse_output=False)
# y_encoded = encoder.fit_transform(y)

# # Flatten (8x8 → 64 features)
# X = X.reshape(len(X), -1)

# # Train/test split
# X_train, X_test, y_train, y_test = train_test_split(
#     X, y_encoded, test_size=0.2, random_state=42
# )

# # Bigger model (~19k params)
# model = tf.keras.Sequential([
#     tf.keras.layers.Dense(128, activation="relu", input_shape=(64,)),
#     tf.keras.layers.Dense(64, activation="relu"),
#     tf.keras.layers.Dense(32, activation="relu"),
#     tf.keras.layers.Dense(10, activation="softmax")
# ])

# model.compile(
#     optimizer="adam",
#     loss="categorical_crossentropy",
#     metrics=["accuracy"]
# )

# history = model.fit(X_train, y_train, epochs=30, batch_size=32,
#                     validation_split=0.1, verbose=1)

# loss, acc = model.evaluate(X_test, y_test, verbose=0)
# print(f"Test Accuracy: {acc:.4f}")

# model.summary()

### Input 28x28
The following won't work as load digit data function only has 8x8 images

In [12]:
# Load dataset
digits = load_digits()
X = digits.images  # shape (n_samples, 28, 28)
y = digits.target.reshape(-1, 1)

# Normalize 0..1
X = X / 16.0  # or 255.0 if your images are 0..255

# Flatten 28x28 → 784 features
X = X.reshape(len(X), -1)

# One-hot encode labels
encoder = OneHotEncoder(sparse_output=False)
y_encoded = encoder.fit_transform(y)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42
)

# Bigger model (~19k params)
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation="relu", input_shape=(784,)),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(32, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax")
])

model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

history = model.fit(X_train, y_train, epochs=30, batch_size=32,
                    validation_split=0.1, verbose=1)

loss, acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {acc:.4f}")

model.summary()


Epoch 1/30


ValueError: in user code:

    File "C:\Users\kukil\.conda\envs\tinyML\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\kukil\.conda\envs\tinyML\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\kukil\.conda\envs\tinyML\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\kukil\.conda\envs\tinyML\lib\site-packages\keras\engine\training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\kukil\.conda\envs\tinyML\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\kukil\.conda\envs\tinyML\lib\site-packages\keras\engine\input_spec.py", line 295, in assert_input_compatibilit

### Let's try loading with 28x28 image

In [13]:
# --- Load MNIST 28x28 dataset ---
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32) / 255.0
X_test  = X_test.astype(np.float32) / 255.0

# Flatten 28x28 → 784
X_train_flat = X_train.reshape(-1, 784)
X_test_flat  = X_test.reshape(-1, 784)

# One-hot encode labels
y_train_onehot = tf.keras.utils.to_categorical(y_train, 10)
y_test_onehot  = tf.keras.utils.to_categorical(y_test, 10)

# --- Build model (~19k params) ---
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation="relu", input_shape=(784,)),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(32, activation="relu"),
    tf.keras.layers.Dense(10, activation="softmax")
])

model.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy"])

# Train model
history = model.fit(X_train_flat, y_train_onehot,
                    epochs=30, batch_size=32,
                    validation_split=0.1, verbose=1)

# Evaluate
loss, acc = model.evaluate(X_test_flat, y_test_onehot, verbose=0)
print(f"Test Accuracy: {acc:.4f}")

model.summary()

# --- Convert to INT8 TFLite ---
def representative_dataset():
    for i in range(1000):  # first 1000 samples for calibration
        yield [X_train_flat[i:i+1].astype(np.float32)]

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

tflite_model = converter.convert()

# Save INT8 model
with open("digits_model_28x28_int8.tflite", "wb") as f:
    f.write(tflite_model)

print("INT8 TFLite model saved. Size:", len(tflite_model), "bytes")

11490434/11490434 [==============================] - 4s 0us/step
Epoch 1/30
1688/1688 [==============================] - 7s 4ms/step - loss: 0.2659 - accuracy: 0.9211 - val_loss: 0.1096 - val_accuracy: 0.9703
Epoch 2/30
1688/1688 [==============================] - 6s 4ms/step - loss: 0.1108 - accuracy: 0.9672 - val_loss: 0.0947 - val_accuracy: 0.9715
Epoch 3/30
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0791 - accuracy: 0.9755 - val_loss: 0.1120 - val_accuracy: 0.9683
Epoch 4/30
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0614 - accuracy: 0.9813 - val_loss: 0.0891 - val_accuracy: 0.9758
Epoch 5/30
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0484 - accuracy: 0.9845 - val_loss: 0.0978 - val_accuracy: 0.9713
Epoch 6/30
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0395 - accuracy: 0.9874 - val_loss: 0.1022 - val_accuracy: 0.9728
Epoch 7/30
1688/1688 [==============================] - 6s 4ms/step - l

C:\Users\kukil\.conda\envs\tinyML\lib\site-packages\tensorflow\lite\python\convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


In [14]:
# Check model layer wise data types
interpreter = tf.lite.Interpreter(model_path="digits_model_28x28_int8.tflite")
interpreter.allocate_tensors()

# Get all tensors
tensor_details = interpreter.get_tensor_details()

print("=== Tensor Data Types ===")
for t in tensor_details:
    print(f"{t['name']:<60} {t['dtype']}")

=== Tensor Data Types ===
serving_default_dense_4_input:0                              <class 'numpy.int8'>
sequential_1/dense_7/BiasAdd/ReadVariableOp                  <class 'numpy.int32'>
sequential_1/dense_7/MatMul                                  <class 'numpy.int8'>
sequential_1/dense_6/BiasAdd/ReadVariableOp                  <class 'numpy.int32'>
sequential_1/dense_6/MatMul                                  <class 'numpy.int8'>
sequential_1/dense_5/BiasAdd/ReadVariableOp                  <class 'numpy.int32'>
sequential_1/dense_5/MatMul                                  <class 'numpy.int8'>
sequential_1/dense_4/BiasAdd/ReadVariableOp                  <class 'numpy.int32'>
sequential_1/dense_4/MatMul                                  <class 'numpy.int8'>
sequential_1/dense_4/MatMul;sequential_1/dense_4/Relu;sequential_1/dense_4/BiasAdd <class 'numpy.int8'>
sequential_1/dense_5/MatMul;sequential_1/dense_5/Relu;sequential_1/dense_5/BiasAdd <class 'numpy.int8'>
sequential_1/dense_6/Mat

In [15]:
# Sanity check
interpreter = tf.lite.Interpreter(model_path="digits_model_28x28_int8.tflite")
interpreter.allocate_tensors()
details = interpreter.get_tensor_details()

float_tensors = [d['name'] for d in details if d['dtype'] == np.float32]
print("Float tensors:", float_tensors if float_tensors else "None ✅ Fully int8")

Float tensors: None ✅ Fully int8


In [16]:
for d in interpreter.get_signature_list():
    print(d)

print("\n=== Operator List ===")
for op in interpreter._get_ops_details():
    print(op["op_name"])


serving_default

=== Operator List ===
FULLY_CONNECTED
FULLY_CONNECTED
FULLY_CONNECTED
FULLY_CONNECTED
SOFTMAX


In [17]:
!xxd -i digits_model_28x28_int8.tflite > digits_model_28x28_int8.h

### Test results

In [4]:
!dir

 Volume in drive C is Windows-SSD
 Volume Serial Number is 143F-0886

 Directory of C:\Users\kukil\Desktop\OpenCV University\tinyML\Tensorflow\Digit Classifier 19k

10/05/2025  10:00 AM    <DIR>          .
09/29/2025  08:35 AM    <DIR>          ..
09/29/2025  09:55 AM    <DIR>          .ipynb_checkpoints
10/05/2025  10:00 AM           283,830 digits_model_19k_int8.h
10/05/2025  09:45 AM            22,688 digits_model_19k_int8.tflite
10/05/2025  09:54 AM            16,839 Digit_Classifier_19k_Param.ipynb
               3 File(s)        323,357 bytes
               3 Dir(s)  322,934,611,968 bytes free


In [18]:
# Load model
interpreter = tf.lite.Interpreter(model_path="digits_model_28x28_int8.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test with one digit image
img = Image.open("../../Dataset/MNIST_ORG/images/mnist_image_45.png").convert("L").resize((28, 28))
# arr = np.array(img, dtype=np.float32)
# arr = (arr / 255.0 * 255 - 128).astype(np.int8)  # same scaling as Arduino
# arr = arr.reshape((1, 28, 28, 1))  # add batch/channel dim if needed

arr = np.array(img, dtype=np.float32).flatten()   # 28*28 = 784
# Scale to match int8 quantization used in Arduino
arr = (arr / 255.0 * 255 - 128).astype(np.int8)
arr = arr.reshape(1, 784)  # batch dimension only

# interpreter.set_tensor(input_details[0]['index'], arr)
# interpreter.invoke()
# output = interpreter.get_tensor(output_details[0]['index'])
# print("Predicted:", np.argmax(output))

interpreter.set_tensor(input_details[0]['index'], arr)
interpreter.invoke()
output = interpreter.get_tensor(output_details[0]['index'])
print("Predicted:", np.argmax(output))

Predicted: 5
